### Money Vs. Happiness

This notebook is used to explore and prep the data for the Streamlit application used in app.py.

In [1]:
import numpy as np
import pandas as pd

### Load the Data

Reference: [Consumer Financial Well-being](https://www.kaggle.com/srolka/consumer-financial-wellbeing-survey)

In [4]:
data = pd.read_csv("resources/data.csv")

/Users/gianmillare/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_

In [5]:
data.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,data
PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,FWB1_4,FWB1_5,FWB1_6,FWB2_1,FWB2_2,FWB2_3,FWB2_4,FSscore,FS1_1,FS1_2,FS1_3,FS1_4,FS1_5,FS1_6,FS1_7,FS2_1,FS2_2,FS2_3,SUBKNOWL1,ACT1_1,ACT1_2,FINGOALS,PROPPLAN_1,PROPPLAN_2,PROPPLAN_3,PROPPLAN_4,MANAGE1_1,MANAGE1_2,MANAGE1_3,MANAGE1_4,SAVEHABIT,FRUGALITY,AUTOMATED_1,AUTOMATED_2,ASK1_1,ASK1_2,SUBNUMERACY2,SUBNUMERACY1,CHANGEABLE,GOALCONF,LMscore,FINKNOWL1,FINKNOWL2,FINKNOWL3,FK1correct,FK2correct,FK3correct,KHscore,KHKNOWL1,KHKNOWL2,KHKNOWL3,KHKNOWL4,KHKNOWL5,KHKNOWL6,KHKNOWL7,KHKNOWL8,KHKNOWL9,KH1correct,KH2correct,KH3correct,KH4correct,KH5correct,KH6correct,KH7correct,KH8correct,KH9correct,ENDSMEET,HOUSING,LIVINGARRANGEMENT,HOUSERANGES,IMPUTATION_FLAG,VALUERANGES,MORTGAGE,SAVINGSRANGES,PRODHAVE_1,PRODHAVE_2,PRODHAVE_3,PRODHAVE_4,PRODHAVE_5,PRODHAVE_6,PRODHAVE_7,PRODHAVE_8,PRODHAVE_9,PRODUSE_1,PRODUSE_2,PRODUSE_3,PRODUSE_4,PRODUSE_5,PRODUSE_6,CONSPROTECT1,CONSPROTECT2,CONSPROTECT3,EARNERS,VOLATILITY,SNAP,MATHARDSHIP_1,MATHARDSHIP_2,MATHARDSHIP_3,MATHARDSHIP_4,MATHARDSHIP_5,MATHARDSHIP_6,COLLECT,REJECTED_1,REJECTED_2,ABSORBSHOCK,BENEFITS_1,BENEFITS_2,BENEFITS_3,BENEFITS_4,BENEFITS_5,FRAUD2,COVERCOSTS,BORROW_1,BORROW_2,SHOCKS_1,SHOCKS_2,SHOCKS_3,SHOCKS_4,SHOCKS_5,SHOCKS_6,SHOCKS_7,SHOCKS_8,SHOCKS_9,SHOCKS_10,SHOCKS_11,SHOCKS_12,MANAGE2,PAIDHELP,HSLOC,PAREDUC,FINSOC2_1,FINSOC2_2,FINSOC2_3,FINSOC2_4,FINSOC2_5,FINSOC2_6,FINSOC2_7,OBJNUMERACY1,ON2correct,ON1correct,MATERIALISM_1,MATERIALISM_2,MATERIALISM_3,CONNECT,HEALTH,SCFHORIZON,DISCOUNT,MEMLOSS,DISTRESS,SELFCONTROL_1,SELFCONTROL_2,SELFCONTROL_3,OUTLOOK_1,OUTLOOK_2,INTERCONNECTIONS_1,INTERCONNECTIONS_2,INTERCONNECTIONS_3,INTERCONNECTIONS_4,INTERCONNECTIONS_5,INTERCONNECTIONS_6,INTERCONNECTIONS_7,INTERCONNECTIONS_8,INTERCONNECTIONS_9,INTERCONNECTIONS_10,PEM,HOUSESAT,SOCSEC1,SOCSEC2,SOCSEC3,LIFEEXPECT,HHEDUC,KIDS_NoChildren,KIDS_1,KIDS_2,KIDS_3,KIDS_4,EMPLOY,EMPLOY1_1,EMPLOY1_2,EMPLOY1_3,EMPLOY1_4,EMPLOY1_5,EMPLOY1_6,EMPLOY1_7,EMPLOY1_8,EMPLOY1_9,RETIRE,MILITARY,Military_Status,agecat,generation,PPEDUC,PPETHM,PPGENDER,PPHHSIZE,PPINCIMP,PPMARIT,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
10350,2,3,5,5,6,55,3,3,3,3,2,3,2,3,2,4,44,3,3,4,3,3,3,4,4,3,4,5,4,3,1,5,4,4,3,4,4,2,4,4,6,0,0,4,3,3,3,4,3,3,1,3,2,1,1,1,1.267,3,3,2,1,1,2,4,2,1,1,1,1,1,1,1,1,1,1,2,1,1,4,0,2,2,6,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,3,2,1,1,2,0,1,1,1,1,1,1,1,0,0,4,0,0,1,0,0,8,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,3,-2,-1,4,0,1,1,1,1,1,1,3,1,0,3,5,4,80,2,3,2,0,4,2,3,3,3,2,0,0,0,0,1,0,1,1,0,0,3,4,1,62,-2,-2,4,-1,0,0,0,0,8,0,0,0,0,0,0,0,1,0,1,0,5,8,1,4,1,1,1,7,3,1,4,8,0,0,0,0,1,0,0.367291878
7740,1,3,6,6,6,51,2,2,3,3,3,4,2,2,2,3,43,3,3,3,3,4,3,2,4,3,2,5,4,3,0,3,2,2,1,4,4,1,4,1,5,0,0,3,2,5,5,2,3,3,1,3,2,1,1,1,-0.57,2,3,3,1,1,2,2,3,1,0,1,0,1,1,1,0,0,1,2,1,2,4,0,2,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,2,1,0,2,2,0,1,1,1,1,1,1,1,0,0,3,1,0,0,0,1,0,3,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,2,0,0,0,0,1,0,0,2,1,1,3,4,3,95,3,3,1,0,3,2,4,3,2,5,0,1,0,0,0,0,0,0,0,0,4,3,-2,-2,66,90,2,1,0,0,0,0,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,2,1,1,2,6,3,1,2,3,0,0,0,0,2,0,1.327560712
13699,1,3,4,3,4,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,5,3,3,1,4,4,4,4,3,3,3,3,5,5,1,1,4,4,4,4,6,3,3,1,3,2,1,1,1,-0.188,3,3,1,1,2,2,2,2,1,1,1,0,1,0,1,0,1,1,1,1,2,3,0,3,2,4,1,1,0,1,1,0,0,0,0,0,0,1,1,1,0,3,3,1,2,3,0,1,1,1,1,1,1,0,0,0,4,1,0,0,0,0,0,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,3,-2,1,3,0,0,0,1,0,0,1,2,0,1,4,4,3,50,3,4,2,0,2,3,3,3,4,4,0,1,0,0,0,0,0,0,0,0,6,3,-2,-2,68,78,3,0,0,0,0,1,2,0,1,0,0,0,0,0,0,0,-2,0,5,3,3,3,2,1,3,6,3,1,4,9,0,0,0,1,2,1,0.835155812
7267,1,3,6,6,6,49,3,3,3,3,3,3,3,3,3,3,42,3,3,3,3,3,3,3,3,3,3,-1,-1,-1,-1,3,3,3,3,4,4,2,4,4,6,-1,-1,-1,-1,-1,-1,-1,-1,2,1,1,2,1,0,1,-1.485,2,2,3,1,2,2,2,-1,1,0,0,0,1,0,1,0,0,1,-1,-1,-1,99,0,-2,-2,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1

In [ ]:
questions = [
    'fpl', # Poverty Status
    'SWB_1', # Are you satisfied with life?
    'FWBscore' # How financially stable are you?
]